# Models from Bower et al. (2024)

## Data generation

This section generates and exports the output.

In [1]:
import copy
import logging

import numpy as np
from pathlib import Path

from atmodeller import debug_logger, debug_file_logger
from atmodeller.constraints import IronWustiteBufferConstraintHirschmann, MassConstraint, SystemConstraints
from atmodeller.core import GasSpecies, Species
from atmodeller.interior_atmosphere import Planet, InteriorAtmosphereSystem
from atmodeller.initial_condition import InitialConditionRegressor, InitialConditionSwitchRegressor, InitialConditionDict, InitialConditionConstant
from atmodeller.plot import Plotter
from atmodeller.utilities import earth_oceans_to_kg

logger = debug_file_logger()
# logger.setLevel(logging.INFO)

Fixed parameters

In [2]:
surface_temperature = 1800
number_of_realisations = 4999

File paths for output

In [3]:
trappist1e_no_sols_path = Path(f"trappist1e_{surface_temperature}K_no_sols_{number_of_realisations}its")
trappist1e_with_sols_path = Path(f"trappist1e_{surface_temperature}K_with_sols_{number_of_realisations}its")

Species to consider. These are initially excluding solubility.

In [4]:
species = Species()
species.append(GasSpecies(formula='H2O'))
species.append(GasSpecies(formula='H2'))
species.append(GasSpecies(formula='O2'))
species.append(GasSpecies(formula='CO'))
species.append(GasSpecies(formula='CO2'))
species.append(GasSpecies(formula='CH4'))
species.append(GasSpecies(formula='N2'))
species.append(GasSpecies(formula='NH3'))
species.append(GasSpecies(formula='S2'))
species.append(GasSpecies(formula='H2S'))
species.append(GasSpecies(formula='SO2'))
species.append(GasSpecies(formula='SO'))
species.append(GasSpecies(formula='Cl2'))
species

[17:37:17 - atmodeller.core                - INFO     ] - Creating GasSpecies H2O using thermodynamic data in JANAF (JANAF name = H2O)
[17:37:17 - atmodeller.core                - INFO     ] - Creating GasSpecies H2 using thermodynamic data in JANAF (JANAF name = H2)
[17:37:17 - atmodeller.core                - INFO     ] - Creating GasSpecies O2 using thermodynamic data in JANAF (JANAF name = O2)
[17:37:17 - atmodeller.core                - INFO     ] - Creating GasSpecies CO using thermodynamic data in JANAF (JANAF name = CO)
[17:37:17 - atmodeller.core                - INFO     ] - Creating GasSpecies CO2 using thermodynamic data in JANAF (JANAF name = CO2)
[17:37:18 - atmodeller.core                - INFO     ] - Creating GasSpecies CH4 using thermodynamic data in JANAF (JANAF name = CH4)
[17:37:18 - atmodeller.core                - INFO     ] - Creating GasSpecies N2 using thermodynamic data in JANAF (JANAF name = N2)
[17:37:18 - atmodeller.core                - INFO     ] - Creat

Species([GasSpecies(formula='H2O', thermodynamic_dataset=<atmodeller.core.ThermodynamicDatasetJANAF object at 0x11fec3550>, name_in_dataset='H2O', _formula=Formula('H2O'), _thermodynamic_data=ThermodynamicDatasetJANAF.ThermodynamicDataForSpecies(species=..., data_source='JANAF', data=<thermochem.janaf.JanafPhase object at 0x11ff08750>), solubility=<atmodeller.core.NoSolubility object at 0x11fabf310>, solid_melt_distribution_coefficient=0, eos=IdealGas(critical_temperature=1, critical_pressure=1, standard_state_pressure=1)),
         GasSpecies(formula='H2', thermodynamic_dataset=<atmodeller.core.ThermodynamicDatasetJANAF object at 0x11f92e5d0>, name_in_dataset='H2', _formula=Formula('H2'), _thermodynamic_data=ThermodynamicDatasetJANAF.ThermodynamicDataForSpecies(species=..., data_source='JANAF', data=<thermochem.janaf.JanafPhase object at 0x11ff748d0>), solubility=<atmodeller.core.NoSolubility object at 0x11faaf6d0>, solid_melt_distribution_coefficient=0, eos=IdealGas(critical_temperat

TRAPPIST-1e planet properties

Mass and radius measurements from Agol et al. 2021; Mantle mass determined assuming same proportion as Earth

In [11]:
trappist1e = Planet(surface_temperature=surface_temperature, mantle_mass = 2.912E24, surface_radius = 5.861E6)

[17:38:54 - atmodeller.interior_atmosphere - INFO     ] - Creating a new planet
[17:38:54 - atmodeller.interior_atmosphere - INFO     ] - mantle_mass = 2.912e+24
[17:38:54 - atmodeller.interior_atmosphere - INFO     ] - mantle_melt_fraction = 1.0
[17:38:54 - atmodeller.interior_atmosphere - INFO     ] - core_mass_fraction = 0.295334691460966
[17:38:54 - atmodeller.interior_atmosphere - INFO     ] - surface_radius = 5861000.0
[17:38:54 - atmodeller.interior_atmosphere - INFO     ] - surface_temperature = 1800
[17:38:54 - atmodeller.interior_atmosphere - INFO     ] - melt_composition = None
[17:38:54 - atmodeller.interior_atmosphere - INFO     ] - planet_mass = 4.1324582957509024e+24
[17:38:54 - atmodeller.interior_atmosphere - INFO     ] - mantle_melt_mass = 2.912e+24
[17:38:54 - atmodeller.interior_atmosphere - INFO     ] - mantle_solid_mass = 0.0
[17:38:54 - atmodeller.interior_atmosphere - INFO     ] - surface_area = 431671430778819.1
[17:38:54 - atmodeller.interior_atmosphere - INFO

Earth planet properties, which are required to scale the bulk volatile inventories for Trappist-1e. Default parameters are Earth so we only need to specify the temperature.

In [14]:
earth = Planet(surface_temperature=surface_temperature)

[18:36:18 - atmodeller.interior_atmosphere - INFO     ] - Creating a new planet
[18:36:18 - atmodeller.interior_atmosphere - INFO     ] - mantle_mass = 4.208261222595111e+24
[18:36:18 - atmodeller.interior_atmosphere - INFO     ] - mantle_melt_fraction = 1.0
[18:36:18 - atmodeller.interior_atmosphere - INFO     ] - core_mass_fraction = 0.295334691460966
[18:36:18 - atmodeller.interior_atmosphere - INFO     ] - surface_radius = 6371000.0
[18:36:18 - atmodeller.interior_atmosphere - INFO     ] - surface_temperature = 1800
[18:36:18 - atmodeller.interior_atmosphere - INFO     ] - melt_composition = None
[18:36:18 - atmodeller.interior_atmosphere - INFO     ] - planet_mass = 5.972e+24
[18:36:18 - atmodeller.interior_atmosphere - INFO     ] - mantle_melt_mass = 4.208261222595111e+24
[18:36:18 - atmodeller.interior_atmosphere - INFO     ] - mantle_solid_mass = 0.0
[18:36:18 - atmodeller.interior_atmosphere - INFO     ] - surface_area = 510064471909788.25
[18:36:18 - atmodeller.interior_atmos

Bulk silicate Earth (BSE) masses of elements in kg 

References:

Hydrogen, Carbon, Nitrogen: Sakuraba et al. 2021

Sulfur: Hirschmann et al. 2016

Chlorine: Kendrick et al. 2017


In [15]:
earth_bse = {"H":{'min': 1.852E20,'max': 1.894E21},
             "C":{'min': 1.767E20, 'max': 3.072E21},
             'S':{'min': 8.416E20, 'max': 1.052E21},
             'N':{'min': 3.493E18, 'max': 1.052E19},
             'Cl': {'min': 7.574E19, 'max': 1.431E20}}

earth_bse

{'H': {'min': 1.852e+20, 'max': 1.894e+21},
 'C': {'min': 1.767e+20, 'max': 3.072e+21},
 'S': {'min': 8.416e+20, 'max': 1.052e+21},
 'N': {'min': 3.493e+18, 'max': 1.052e+19},
 'Cl': {'min': 7.574e+19, 'max': 1.431e+20}}

Compute the reservoir sizes for TRAPPIST 1-e, assuming the same ppmw as Earth:

In [16]:
trappist1e_bse = copy.deepcopy(earth_bse)
mass_scale_factor = trappist1e.mantle_mass / earth.mantle_mass

for element, values in trappist1e_bse.items():
    trappist1e_bse[element] = {key: value*mass_scale_factor for key, value in values.items()}

trappist1e_bse

{'H': {'min': 1.2815326128149148e+20, 'max': 1.3105954474467865e+21},
 'C': {'min': 1.2227149712980315e+20, 'max': 2.1257387616454742e+21},
 'S': {'min': 5.8236384824245807e+20, 'max': 7.279548103030726e+20},
 'N': {'min': 2.417059080217331e+18, 'max': 7.279548103030726e+18},
 'Cl': {'min': 5.240997845280866e+19, 'max': 9.902122942430578e+19}}

In [28]:

trappist1e_bse['H']['avg']=np.mean((trappist1e_bse['H']['min'], trappist1e_bse['H']['max']))
trappist1e_bse['C']['avg']=np.mean((trappist1e_bse['C']['min'], trappist1e_bse['C']['max']))
trappist1e_bse['N']['avg']=np.mean((trappist1e_bse['N']['min'], trappist1e_bse['N']['max']))
trappist1e_bse['S']['avg']=np.mean((trappist1e_bse['S']['min'], trappist1e_bse['S']['max']))
trappist1e_bse['Cl']['avg']=np.mean((trappist1e_bse['Cl']['min'], trappist1e_bse['Cl']['max']))

print('N:', trappist1e_bse['N']['avg'])
print('S:', trappist1e_bse['S']['avg'])
print('Cl:', trappist1e_bse['Cl']['avg'])

N: 4.848303591624028e+18
S: 6.551593292727653e+20
Cl: 7.571560393855723e+19


Now set up the main driver of the Monte Carlo (MC) approach. This establishes the ranges over which we sample certain properties.

In [27]:
def monte_carlo(interior_atmosphere: InteriorAtmosphereSystem, bse: dict, number_of_realisations:int=100):
    """Monte Carlo driver
    
    Args:
        interior_atmosphere: An interior-atmosphere system
        bse: Dictionary of element masses, fixed to the ppmw of the bulk silicate Earth
        number_of_realisation: Number of simulations to perform
    """

    # Parameters are normally distributed between bounds.
    number_ocean_moles = np.random.uniform(0.1, 10, number_of_realisations)
    ch_ratios = np.random.uniform(0.1, 10, number_of_realisations)
    fo2_shifts = np.random.uniform(-5, 5, number_of_realisations)

    for realisation in range(number_of_realisations):

        mass_H = earth_oceans_to_kg(number_ocean_moles[realisation])
        mass_C = ch_ratios[realisation] * mass_H
        constraints = SystemConstraints([
            MassConstraint(species="H", value=mass_H),
            MassConstraint(species="C", value=mass_C),
            MassConstraint(species="N", value=trappist1e_bse['N']['avg']),
            MassConstraint(species="S", value=trappist1e_bse['S']['avg']),
            MassConstraint(species="Cl", value=trappist1e_bse['Cl']['avg']),
            IronWustiteBufferConstraintHirschmann(log10_shift=fo2_shifts[realisation])
        ])

        # Extra quantities to write to the output
        # For example, it's often helpful to have the constraints expressed in a more convenient
        # form for analysis and plotting.
        extra = {'fO2_shift': fo2_shifts[realisation], 'C/H ratio': ch_ratios[realisation],
            'Number of ocean moles': number_ocean_moles[realisation]}

        interior_atmosphere.solve(constraints, extra_output=extra, tol=1e-5)

### Trappist 1-e

### CHONSCl with no solubilities

If we have no initial data to begin from, start with a dictionary of initial guesses and then switch to a regressor.

In [ ]:
# values = {'O2': 1E-8, 'H2': 100, 'H2O': 100, 'CO2': 100, 'CO': 500, 'S2': 1}
# values = {'O2': 1E9, 'H2': 1.0, 'H2O': 1, 'CO2': 1, 'CO': 5, 'S2': 1}
initial_condition_start = InitialConditionConstant()
# initial_condition = InitialConditionSwitchRegressor(initial_condition_start, fit=True, fit_batch_size=50, partial_fit=True, partial_fit_batch_size=50)

Otherwise, if data is available then train a network and use this from the start.

In [ ]:
trappist1e_with_sols_200_its_path = Path(f"trappist1e_{surface_temperature}K_no_sols_1095its")
initial_condition = InitialConditionRegressor.from_pickle(trappist1e_with_sols_200_its_path.with_suffix(".pkl"), fit=False, fit_batch_size=100, partial_fit=True, partial_fit_batch_size=100)

In [ ]:
trappist1e_no_sols = InteriorAtmosphereSystem(species=species, initial_condition=initial_condition_start, planet=trappist1e)
monte_carlo(trappist1e_no_sols, trappist1e_bse, number_of_realisations=number_of_realisations)

In [ ]:
trappist1e_no_sols.output(file_prefix=trappist1e_no_sols_path, to_pickle=True, to_excel=True)

### CHONSCl with solubilities

This will set all the solubilities (where possible) to conform to a basaltic composition.

In [ ]:
trappist1e.melt_composition = 'basalt'

The pickle file output for the model without solubilities (above) is used to inform the initial condition. But because solubilities can strongly affect the solution, we use the `species_fill` dictionary to modify some initial guesses.

In [ ]:
species_fill = {'Cl2': 1E-6, 'H2': 1, 'H2O': 1, 'S2': 1E-6, 'SO2': 1E-6, 'SO': 1E-6}
initial_condition = InitialConditionRegressor.from_pickle(trappist1e_no_sols_path.with_suffix(".pkl"), species_fill=species_fill, fit=True, fit_batch_size=100, partial_fit=True, partial_fit_batch_size=50)

Alternatively, here we can instead use previous output to train the network and either run with a tighter tolerance and/or for more iterations

In [ ]:
trappist1e_with_sols_200_its_path = Path(f"trappist1e_{surface_temperature}K_with_sols_200its")
initial_condition = InitialConditionRegressor.from_pickle(trappist1e_with_sols_200_its_path.with_suffix(".pkl"), fit=False, fit_batch_size=100, partial_fit=True, partial_fit_batch_size=50)

In [ ]:
trappist1e_with_sols = InteriorAtmosphereSystem(species=species, initial_condition=initial_condition, planet=trappist1e)
monte_carlo(trappist1e_with_sols, trappist1e_bse, number_of_realisations=number_of_realisations)

In [ ]:
trappist1e_with_sols.output(file_prefix=trappist1e_with_sols_path, to_pickle=True, to_excel=True)

## Data plotting

You can just run this import rather than having to return to the top of the notebook to do the required imports

In [ ]:
from atmodeller.plot import Plotter
import matplotlib.pyplot as plt
from pathlib import Path

import numpy as np

This section plots the output. This reads the data exported by the data generation section above.

### TRAPPIST 1-e with no solubilities

In [ ]:
# Below are used to set the filename of the data to plot
plot_iterations = 5000
surface_temperature = 1800
trappist1e_no_sols_1000_its_path = Path(f"trappist1e_{surface_temperature}K_no_sols_{plot_iterations}its")

plotter_no_sols = Plotter.read_pickle(trappist1e_no_sols_1000_its_path.with_suffix('.pkl'))

major_species = ("CO", "CO2", "H2", "H2O")
major_species_str = "_".join(major_species)

elements = ("C", "H", "O", "N")
elements_str = "_".join(elements)

categories: tuple[str,...] = ("fO2", "CH", "H")

Major species

In [ ]:
for category in categories:
    plotter_no_sols.species_pairplot(major_species, mass_or_moles='moles', category=category)
    plt.savefig(f"trappist1e_no_sols_{major_species_str}_by_moles_{category}.pdf", format='pdf')
    plotter_no_sols.species_pairplot(major_species, mass_or_moles='mass', category=category)
    plt.savefig(f"trappist1e_no_sols_{major_species_str}_by_mass_{category}.pdf", format='pdf')

In [ ]:
# The axes adjustments are for N, which has a very low abundance.
N_min = 0
N_max = 0.25
N_step = 0.05
N_ticks = np.arange(N_min, N_max+0.001, N_step)
N_ticksx = np.arange(N_min, N_max+0.001, 0.1)

for category in categories:
    ax = plotter_no_sols.species_pairplot(elements, mass_or_moles='moles', category=category)
    ax.axes[-1][-1].set_xlim(N_min, N_max)
    ax.axes[-1][0].set_ylim(N_min, N_max)
    ax.axes[-1][-1].set_xticks(N_ticksx)
    ax.axes[-1][0].set_yticks(N_ticks)
    plt.savefig(f"trappist1e_no_sols_{elements_str}_by_moles_{category}.pdf", format='pdf')
    ax = plotter_no_sols.species_pairplot(elements, mass_or_moles='mass', category=category)
    ax.axes[-1][-1].set_xlim(N_min, N_max)
    ax.axes[-1][0].set_ylim(N_min, N_max)
    ax.axes[-1][-1].set_xticks(N_ticksx)
    ax.axes[-1][0].set_yticks(N_ticks)
    plt.savefig(f"trappist1e_no_sols_{elements_str}_by_mass_{category}.pdf", format='pdf')

Ratio plots. These are just to sanity check that without considering solubilities the ratios in the atmosphere must correspond exactly to the ratios in the interior (melt).

In [ ]:
plotter_no_sols.ratios_pairplot(['atmosphere','total'], mass_or_moles='moles')
plt.savefig(f"trappist1e_no_sols_ratios_by_moles.pdf", format='pdf')

In [ ]:
plotter_no_sols.ratios_pairplot(['atmosphere','total'], mass_or_moles='mass')
plt.savefig(f"trappist1e_no_sols_ratios_by_mass.pdf", format='pdf')

### Trappist 1-e with solubilities

In [ ]:
trappist1e_with_sols_1000_its_path = Path(f"trappist1e_{surface_temperature}K_with_sols_1000its")
plotter_with_sols = Plotter.read_pickle(trappist1e_with_sols_1000_its_path.with_suffix('.pkl'))

In [ ]:
plotter_with_sols.species_pairplot()

In [ ]:
plotter_with_sols.species_pairplot(("H2O", "H2", "CO2", "CO"))

In [ ]:
plotter_with_sols.ratios_pairplot(['atmosphere','total'])